In [9]:
import requests
import pandas as pd

API_KEY = '5581d8d29db643f6893fe91896b724ac'
BASE_URL = 'https://newsapi.org/v2/everything'

# Prompt the user for company name and keyword(s)
company = input("Enter the company you want to search for: ").strip()
user_keywords = input("Enter the keyword(s) you want to filter by (comma-separated): ").strip()

# Convert the comma-separated keywords into a list and strip extra spaces
keywords = [kw.strip().lower() for kw in user_keywords.split(',') if kw.strip()]

params = {
    'q': company,  
    'language': 'en',
    'sortBy': 'relevance',
    'apiKey': API_KEY,
}

def fetch_and_filter_articles():
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        
        filtered_articles = []
        for article in articles:
            # Combine content and description, convert to lowercase
            content = (article.get('content', '') or '') + ' ' + (article.get('description', '') or '')
            content = content.lower()
            # Check if any of the provided keywords are in the content
            if any(keyword in content for keyword in keywords):
                filtered_articles.append(article)
        
        if filtered_articles:
            print('\nFiltered Articles on', company, ":\n")
            for idx, article in enumerate(filtered_articles, 1):
                print(f"Article {idx}:")
                print(f"Title: {article['title']}")
                print(f"URL: {article['url']}")
                print(f"Description: {article.get('description', 'No description available.')}\n")
        else:
            print("\nNo articles found matching the specified keywords for this company.")
    
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

fetch_and_filter_articles()


Enter the company you want to search for:  Chevron
Enter the keyword(s) you want to filter by (comma-separated):  Permian Basin



Filtered Articles on Chevron :

Article 1:
Title: Chevron Cuts Permian Capex for 2025
URL: https://oilprice.com/Latest-Energy-News/World-News/Chevron-Cuts-Permian-Capex-for-2025.html
Description: Chevron has reduced the amount of capital expenditure it will allocate for its operations in the Permian Basin next year, the company said in an update. The company said it planned to spend between $4.5 billion and $5 billion on production in the Permian, “as…

Article 2:
Title: Chevron Cuts Permian Capex for 2025
URL: https://biztoc.com/x/af354a14c48ed5a0
Description: Chevron Cuts Permian Capex for 2025
 
By Irina Slav of OilPrice.com
Chevron has reduced the amount of capital expenditure it will allocate for its operations in the Permian Basin next year, the company said in an update.
The company said it planned to spend b…

